In [1]:
import os
import re
import nltk
import spacy
import random
import neuralcoref
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

/Users/johnchen/opt/anaconda3/envs/SP/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
date_suffices = [
#     '20210501_20211231', '20210101_20210430', '20200101_20201231',
#                  '20190101_20191231', '20180101_20181231', '20170101_20171231',
                 '20160101_20161231'
]
for date_suffix in date_suffices:
    submission_filtered_df = pd.read_csv('../data/filtered_q/submission/casual_conv_submissions_{}.csv'.format(date_suffix))
    comment_filtered_df = pd.read_csv('../data/filtered_q/comment/casual_conv_comments_{}.csv'.format(date_suffix))
    comment_filtered_parent_ids = comment_filtered_df['parent_id'].tolist()
    comment_filtered_parent_ids = [x[3:] for x in comment_filtered_parent_ids]
    print(comment_filtered_parent_ids)
    submission_filtered_df = submission_filtered_df[submission_filtered_df['id'].isin(comment_filtered_parent_ids)]
    print(submission_filtered_df.shape)
    final_cols = ['sub_id', 'sub_summarized', 'sub_from', 'sub_text', 'sub_root', 'sub_length',
                  'com_id', 'com_summarized', 'com_text', 'com_length']
    final_dict = {col: [] for col in final_cols}
    for i in tqdm(range(submission_filtered_df.shape[0])):
        sub_id = submission_filtered_df.iloc[i]['id']
        sub_summarized = submission_filtered_df.iloc[i]['summarized']
        sub_from = submission_filtered_df.iloc[i]['from']
        sub_text = submission_filtered_df.iloc[i]['text']
        sub_root = submission_filtered_df.iloc[i]['root']
        sub_length = submission_filtered_df.iloc[i]['length']

        comment_filtered_df_sub = comment_filtered_df[comment_filtered_df['parent_id'] == 't3_' + sub_id]
        for j in range(comment_filtered_df_sub.shape[0]):
            final_dict['sub_id'].append(sub_id)
            final_dict['sub_summarized'].append(sub_summarized)
            final_dict['sub_from'].append(sub_from)
            final_dict['sub_text'].append(sub_text)
            final_dict['sub_root'].append(sub_root)
            final_dict['sub_length'].append(sub_length)
            final_dict['com_id'].append(comment_filtered_df_sub.iloc[j]['id'])
            final_dict['com_summarized'].append(comment_filtered_df_sub.iloc[j]['summarized'])
            final_dict['com_text'].append(comment_filtered_df_sub.iloc[j]['text'])
            final_dict['com_length'].append(comment_filtered_df_sub.iloc[j]['length'])
    final_df = pd.DataFrame(final_dict)
    print(final_df.shape)
    final_df.to_csv('../data/matched_q/casual_conv_{}.csv'.format(date_suffix), index = False)

['5la2d8', '5lc4hd', '5lcee8', '5lcamq', '5lcgdm', '5lce7x', '5lcfk7', '5lce7x', '5la2d8', '5lcfk7', '5lce7x', '5lc4hd', '5lcfk7', '5l9zl2', '5lcamq', '5lce7x', '5lc2z7', '5la2d8', '5lcdab', '5lcgdm', '5lcgdm', '5l9xka', '5l9xka', '5la2d8', '5la2d8', '5l3q41', '5l9xka', '5lce7x', '5la2d8', '5la2d8', '5lak6m', '5lak6m', '5l93ar', '5l9xka', '5lcfk7', '5la2d8', '5lc4hd', '5lbh9g', '5la2d8', '5lc2z7', '5la2d8', '5lcgdm', '5l9zl2', '5lb44z', '5lcdab', '5laucv', '5lcfk7', '5la2d8', '5lcgdm', '5lbezd', '5lbz35', '5lcgdm', '5l93ar', '5lcdab', '5la2d8', '5lcfk7', '5la2d8', '5lcdab', '5lam6v', '5l9xka', '5la2d8', '5la2d8', '5lce7x', '5la2d8', '5lbh9g', '5lce7x', '5l9xka', '5lb4t5', '5lcdab', '5la2d8', '5l9zl2', '5la2d8', '5la2d8', '5lce7x', '5la2d8', '5la2d8', '5la2d8', '5l9xka', '5lcfk7', '5la2d8', '5lby1s', '5la2d8', '5la2d8', '5la2d8', '5l3c40', '5lbh9g', '5la5p8', '5l9xka', '5la2d8', '5la2d8', '5la2d8', '5lbh9g', '5la2d8', '5la2d8', '5lbgw5', '5lbgw5', '5lb4t5', '5lby1s', '5lalwg', '5la2d8',